In [10]:
import pandas as pd
import numpy as np
import pickle
import os
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                        roc_curve, auc, roc_auc_score, log_loss)
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import LeaveOneOut
from essentials import complete_preprocessing_pipeline
import copy
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from typing import Tuple, Dict
from essentials import normalization
from feature_sets_center_less import GenerateFeatures
from two_class_ncv_selectk import ModifiedNestedCVOptimizer



In [2]:
with open('../data/df_dict_imu.pkl', 'rb') as f:
    imu_dict = pickle.load(f)
with open('../data/df_dict_urineestimate_method1.pkl', 'rb') as f:
    urine_estimates_dict = pickle.load(f)
with open('../data/df_minze_dict.pkl', 'rb') as f:
    ground_truth_dict = pickle.load(f)

In [3]:
del imu_dict['subj_9_void4']
del imu_dict['subj_11_void2']

In [4]:
data_dict = copy.deepcopy(imu_dict)
labelled_imu_dict = complete_preprocessing_pipeline(data_dict, ground_truth_dict, 
                                target_fs=60,normalize_data=False, use_three_classes=False)

Step 1: Resampling data to 60 Hz...


Step 2: Processing each instance: 100%|██████████| 41/41 [00:00<00:00, 451.96it/s]


In [5]:
# Add experiment_id to each dataframe and append to imu_dict
imu_list = []
for i, key in enumerate(labelled_imu_dict.keys()):
    df  = labelled_imu_dict[key]
    df['experiment_id'] = i + 1

    imu_list.append(df)

In [6]:
# Concatenate all the dataframes in imu_dict into a single dataframe
main_df = pd.concat(imu_list, ignore_index=True)

In [7]:
# Define window configurations (same as your pipeline)
window_configs = [
    (1, 0.0), (1, 0.5), (1, 0.8),
    (2, 0.0), (2, 0.5), (2, 0.8),
    (3, 0.0), (3, 0.5), (3, 0.8),
    (4, 0.0), (4, 0.5), (4, 0.8),
    (5, 0.0), (5, 0.5), (5, 0.8)
]

def dataframe_to_dict_by_experiment(df: pd.DataFrame) -> dict:
    """Convert DataFrame to dictionary with experiment_id as key"""
    experiment_dict = {}
    for exp_id in df['experiment_id'].unique():
        exp_data = df[df['experiment_id'] == exp_id].copy()
        exp_data = exp_data.drop(columns=['experiment_id'])
        # IMPORTANT: Reset index to avoid KeyError in feature extraction
        exp_data = exp_data.reset_index(drop=True)
        experiment_dict[f'exp_{exp_id}'] = exp_data
    return experiment_dict

def extract_features_from_dict(data_dict: dict, window_size: float, overlap: float) -> pd.DataFrame:
    """Extract features from dictionary of DataFrames"""
    all_features = []
    
    for exp_key, df in data_dict.items():
        actual_exp_id = int(exp_key.split('_')[1])
        
        analyzer = GenerateFeatures(fs=60, window_duration=window_size, overlap=overlap)
        features, _ = analyzer.analyze_multi_axis_imu(df)
        
        table = analyzer.create_summary_table()
        table['experiment_id'] = actual_exp_id
        all_features.append(table)
    
    return pd.concat(all_features, ignore_index=True)


In [8]:
# Store results for all window configurations
all_global_norm_results = {}

In [9]:
n_outer_splits = 5
n_inner_splits = 3

outer_cv = StratifiedGroupKFold(n_splits=n_outer_splits, shuffle=True, random_state=42)
inner_cv = StratifiedGroupKFold(n_splits=n_inner_splits, shuffle=True, random_state=42)

# Loop through each window configuration
for window_size, overlap in window_configs:
    print(f"\n{'='*80}")
    print(f"PROCESSING: {window_size}s window, {overlap} overlap")
    print(f"{'='*80}")
    
    config_fold_results = []

    # Split dictionary into training and testing sets based on void instances
    for fold_id, (train_id, test_id) in enumerate(outer_cv.split(main_df, y=main_df['label'], groups=main_df['experiment_id'])):
        print(f"Fold {fold_id + 1}")
        data_train, data_test = main_df.iloc[train_id], main_df.iloc[test_id]
        _, _ = main_df['label'].iloc[train_id], main_df['label'].iloc[test_id]
        groups_train, groups_test = main_df['experiment_id'].iloc[train_id], main_df['experiment_id'].iloc[test_id]

        
        # Apply global normalization
        data_train_norm, data_test_norm = normalization(data_train, data_test)

        print(f"✓ Applied global normalization")
        print(f"Train experiments: {sorted(groups_train.unique())}")
        print(f"Test experiments: {sorted(groups_test.unique())}")
    
        # Convert normalized DataFrames to dictionaries for feature extraction
        train_dict = dataframe_to_dict_by_experiment(data_train_norm)
        test_dict = dataframe_to_dict_by_experiment(data_test_norm)
        
        # Extract features
        print(f"Extracting features...")
        train_features = extract_features_from_dict(train_dict, window_size, overlap)
        test_features = extract_features_from_dict(test_dict, window_size, overlap)
        
        print(f"Train features shape: {train_features.shape}")
        print(f"Test features shape: {test_features.shape}")
        
        # Store this fold's results
        fold_result = {
            'fold': fold_id + 1,
            'window_size': window_size,
            'overlap': overlap,
            'train_features': train_features,
            'test_features': test_features,
            'original_train_groups': groups_train,
            'original_test_groups': groups_test
        }
        config_fold_results.append(fold_result)
        
    # Store results for this configuration - for each window size and overlap, we get 5 fold results
    overlap_str = 'no' if overlap == 0.0 else str(overlap)
    config_key = f"{window_size}s_{overlap_str}"
    all_global_norm_results[config_key] = config_fold_results
    
    print(f"\n✓ Completed {config_key}: {len(config_fold_results)} folds")
    
print(f"\n{'='*80}")
print("FEATURE EXTRACTION COMPLETED FOR ALL CONFIGURATIONS")
print(f"{'='*80}")

# Show summary
for config_key, fold_results in all_global_norm_results.items():
    print(f"\n{config_key}:")
    for fold_result in fold_results:
        fold_id = fold_result['fold']
        train_shape = fold_result['train_features'].shape
        test_shape = fold_result['test_features'].shape
        print(f"  Fold {fold_result['fold']}: Train{train_shape}, Test{test_shape}")
        
        # Save train and test features to CSV for inspection
        fold_result['train_features'].to_csv(
            f'/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/global_normalization/extracted_features/train_{config_key}_fold{fold_id}.csv', 
            index=False
        )
        fold_result['test_features'].to_csv(
            f'/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/global_normalization/extracted_features/test_{config_key}_fold{fold_id}.csv', 
            index=False
        )
        



PROCESSING: 1s window, 0.0 overlap
Fold 1
✓ Applied global normalization
Train experiments: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(22), np.int64(24), np.int64(25), np.int64(26), np.int64(28), np.int64(30), np.int64(31), np.int64(32), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(41)]
Test experiments: [np.int64(13), np.int64(20), np.int64(21), np.int64(23), np.int64(27), np.int64(29), np.int64(33), np.int64(39), np.int64(40)]
Extracting features...


Analyzing: 100%|██████████| 6/6 [00:01<00:00,  3.24it/s]


Train features shape: (1782, 107)
Test features shape: (616, 107)
Fold 2
✓ Applied global normalization
Train experiments: [np.int64(1), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(15), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(33), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41)]
Test experiments: [np.int64(2), np.int64(3), np.int64(14), np.int64(16), np.int64(26), np.int64(32), np.int64(34)]
Extracting features...


Analyzing: 100%|██████████| 6/6 [00:00<00:00, 16.59it/s]


Train features shape: (2064, 107)
Test features shape: (334, 107)
Fold 3
✓ Applied global normalization
Train experiments: [np.int64(1), np.int64(2), np.int64(3), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(20), np.int64(21), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(29), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41)]
Test experiments: [np.int64(4), np.int64(12), np.int64(17), np.int64(18), np.int64(19), np.int64(22), np.int64(28), np.int64(30), np.int64(36)]
Extracting features...


...

Train features shape: (1735, 107)
Test features shape: (499, 107)
Fold 4
✓ Applied global normalization
Train experiments: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(7), np.int64(9), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(32), np.int64(33), np.int64(34), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40)]
Test experiments: [np.int64(6), np.int64(8), np.int64(10), np.int64(15), np.int64(31), np.int64(35), np.int64(41)]
Extracting features...


Analyzing: 100%|██████████| 6/6 [00:00<00:00, 15.60it/s]


Train features shape: (1896, 107)
Test features shape: (338, 107)
Fold 5
✓ Applied global normalization
Train experiments: [np.int64(2), np.int64(3), np.int64(4), np.int64(6), np.int64(8), np.int64(10), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(39), np.int64(40), np.int64(41)]
Test experiments: [np.int64(1), np.int64(5), np.int64(7), np.int64(9), np.int64(11), np.int64(24), np.int64(25), np.int64(37), np.int64(38)]
Extracting features...


Analyzing: 100%|██████████| 6/6 [00:00<00:00, 15.16it/s]


Train features shape: (1723, 107)
Test features shape: (511, 107)

✓ Completed 5s_0.8: 5 folds

FEATURE EXTRACTION COMPLETED FOR ALL CONFIGURATIONS

1s_no:
  Fold 1: Train(1782, 107), Test(616, 107)
  Fold 2: Train(2064, 107), Test(334, 107)
  Fold 3: Train(1863, 107), Test(535, 107)
  Fold 4: Train(2032, 107), Test(366, 107)
  Fold 5: Train(1851, 107), Test(547, 107)

1s_0.5:
  Fold 1: Train(3555, 107), Test(1227, 107)
  Fold 2: Train(4116, 107), Test(666, 107)
  Fold 3: Train(3714, 107), Test(1068, 107)
  Fold 4: Train(4052, 107), Test(730, 107)
  Fold 5: Train(3691, 107), Test(1091, 107)

1s_0.8:
  Fold 1: Train(8859, 107), Test(3057, 107)
  Fold 2: Train(10258, 107), Test(1658, 107)
  Fold 3: Train(9253, 107), Test(2663, 107)
  Fold 4: Train(10099, 107), Test(1817, 107)
  Fold 5: Train(9195, 107), Test(2721, 107)

2s_no:
  Fold 1: Train(882, 107), Test(306, 107)
  Fold 2: Train(1023, 107), Test(165, 107)
  Fold 3: Train(923, 107), Test(265, 107)
  Fold 4: Train(1008, 107), Test(180

# Nested CV with pre-split data